<a href="https://colab.research.google.com/github/Ashitpatel001/cookbook/blob/langgraph-research-agent/examples/Agents/Gemini_LangGraph_Research_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# @title Copyright 2025 Google LLC {"display-mode":"form"}
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Build a Research Agent with Gemini and LangGraph

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Agents/Gemini_LangGraph_Research_Agent.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>
  </td>
</table>

This tutorial demonstrates how to use Gemini 1.5 Flash and LangGraph to create a Research Agent that can plan, search the web, and write reports.

Basic chains, such as those found in LangChain, carry out steps in a linear order (A --> B --> C), but they frequently break down when real-world data is incomplete or messy. This is resolved by LangGraph's introduction of cycles, which let your agent "loop back," retry steps that didn't work, or adjust its strategy in light of fresh data.


Note: The Research Agent pattern is implemented in an interactive, simplified manner in this tutorial. Check out the official Gemini Fullstack LangGraph Quickstart for a full-stack implementation that is ready for production, complete with React frontend and Docker support.
## Prerequisites

To run this tutorial, you will need:
- A **Google AI Studio API Key** (Get one [here](https://aistudio.google.com/)).
- A **Tavily API Key** for web search (Get one [here](https://tavily.com/)).

In [39]:
!pip install -U -q langgraph langchain-google-genai langchain-community tavily-python

In [40]:
import os
from google.colab import userdata

# Retrieve keys from Colab Secrets
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('Langgraph')
    os.environ["TAVILY_API_KEY"] = userdata.get('Tavily')
except Exception as e:
    print(" Error: Please set 'GOOGLE_API_KEY' and 'TAVILY_API_KEY' in Colab Secrets.")

## Step 1: Define the Agent State and Tools

We start by defining the **State** (the memory structure passed between nodes) and initializing our tools.

We use **Gemini 2.5 Flash** for its speed and **Tavily** for optimized search results.

In [41]:
from typing import Annotated, List, TypedDict
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults

# Define the Agent State (Memory)
class AgentState(TypedDict):
    topic: str
    plan: List[str]
    content: List[str]
    final_report: str


class Plan(BaseModel):

    steps: List[str] = Field(description="List of search queries to run.")

#initialize the model and search tool
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0) # We use Gemini 2.5 Flash for speed and structured output capabilities
search_tool = TavilySearchResults(max_results=1)

## Step 2: Create the Agent Nodes

Here we define the three distinct roles our agent will perform:
1.  **Planner:** Uses Gemini's structured output to break the topic into search queries.
2.  **Researcher:** Executes the queries using the Tavily API.
3.  **Writer:** Synthesizes the findings into a report, streaming the output for a real-time effect.

In [42]:
# PLANNER
def plan_node(state: AgentState):
    print(f"PLANNING: {state['topic']}")
    planner = model.with_structured_output(Plan) # Pydantic schema for the model to invoke the structured output
    result = planner.invoke(f"Create a step-by-step search plan for: {state['topic']}")
    return {"plan": result.steps}  #Displays the steps only

# RESEARCHER
def research_node(state: AgentState):
    print(f" RESEARCHING ")
    content = []
    for step in state['plan']:
        result = search_tool.invoke(step)  #Tavily tool will invoke the plan and start researching
        content.append(str(result))
    return {"content": content}

# WRITER
def write_node(state: AgentState):
  print(f" WRITING ")
  prompt = (
        f"Write a detailed article about '{state['topic']}' using this research: {state['content']}. "
        f"Focus on the content. Do NOT include a 'Date', 'Prepared For', or 'Prepared By' section. "
        f"Start directly with the Title and Introduction."
    )

  full_report = ""

  #Stream output for real time effect
  for chunk in model.stream(prompt):
        text = chunk.content    # Here we will print every chunk as soon as it generates
        print(text,end="", flush=True)
        full_report += text

  print("\n")
  return {"final_report": full_report}



## Step 3: Build the Graph with "Human-in-the-Loop"

We connect the nodes using **LangGraph**.

Key features here:
* **MemorySaver()**: Persists the state between pauses.
* **interrupt_before=["researcher"]**: This is the "Manager Mode." The graph will **PAUSE** after planning, allowing you to approve the plan before the agent spends time searching.

In [43]:
from langgraph.checkpoint.memory import MemorySaver #Import for memory
from langgraph.graph import StateGraph ,START, END

# Create the Graph
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("planner", plan_node)
workflow.add_node("researcher", research_node)
workflow.add_node("writer", write_node)

# Add Edges
workflow.add_edge(START , "planner")
workflow.add_edge("planner", "researcher")
workflow.add_edge("researcher", "writer")
workflow.add_edge("writer", END)

#Compile
memory = MemorySaver() #It will remember the pauses
app = workflow.compile(
    checkpointer=memory,
    interrupt_before=["researcher"] # Pause here!
)

## Step 4: Execution till Planning

Run the cell below to generate a plan. The agent will **pause** automatically.

In [56]:
# GENERATE PLAN
thread = {"configurable": {"thread_id": "1"}}

#Enter your query here
user_input = {"topic": "the mystery of pyramid giza"}

# Run until the pause
for event in app.stream(user_input, thread, stream_mode='values'):
    result = event

# Print the plan for the user to see
print(f"Current plan: {result['plan']}")
print("\n PAUSED: Waiting for your approval....")
print("If the plan matches your expectations Execute the next cell to continue the breakdown")

PLANNING: the mystery of pyramid giza
Current Plan: ['mystery of Giza pyramids', 'how were Giza pyramids built', 'purpose of Giza pyramids', 'theories about Giza pyramid construction', 'unexplained features Giza pyramids', 'ancient Egyptian beliefs about pyramids', 'discoveries inside Giza pyramids']

 PAUSED: Waiting for your approval....
If the plan matches your expectations Execute the next cell to continue the breakdown


## Step 5: Approval after planning

If you are happy with the plan above, run this cell to **resume** the agent. It will execute the searches and write the final report.

In [57]:
from IPython.display import clear_output, Markdown

print(" APPROVED. RESUMING GRAPH ")

# Passing None resumes execution from the paused state
for event in app.stream(None, thread, stream_mode="values"):
    if 'final_report' in event and event['final_report']:
        final_content = event['final_report']

clear_output()

# Show only the beautiful final report
display(Markdown(final_content))

## The Enduring Enigma of the Giza Pyramids

For millennia, the Pyramids of Giza have stood as a testament to ancient ambition and a profound mystery. Perched on the outskirts of modern-day Cairo, this monumental necropolis, dating back to Egypt's Old Kingdom era some 4,500 years ago, comprises three towering pyramids: the Great Pyramid of Khufu, the Pyramid of Khafre, and the Pyramid of Menkaure. Among them, the Great Pyramid, built for Pharaoh Khufu and completed around 2560 BCE, holds a singular distinction as the only remaining marvel of the original Seven Wonders of the Ancient World. Their sheer scale and precision continue to baffle and inspire, leading generations to ponder the profound questions: how were they built, and what secrets do they still hold?

### The Enduring Mystery of Construction

Despite centuries of inquiry and extensive archaeological work, the precise methods employed in the construction of the Giza Pyramids remain one of the ancient world's greatest enduring mysteries. Archaeologists have unearthed clues from excavations and ancient records, providing insights into the tools and techniques likely used. King Khufu, the second pharaoh of the Fourth Dynasty, reigned between 2585 BC and 2560 BC, during which time he led the estimated 20-year construction of his colossal tomb.

Numerous theories attempt to explain this feat of engineering. The most widely accepted is the **Ramp Theory**, which posits the use of massive ramp systems to move and position the enormous stone blocks, some weighing many tons. Another intriguing proposition is the **Limestone Concrete Theory**, which suggests that the ancient Egyptians might have created a form of concrete mix from limestone and clay, pouring it into wooden molds to form the blocks in situ. While archaeological research continues to shed new light on workforce organization and construction techniques, certainty remains elusive. Fringe theories, ranging from extraterrestrial involvement to other pseudoarchaeological explanations, also abound, but these lack sufficient proof and are not accepted by mainstream experts.

### Purpose Beyond the Tomb

Beyond their awe-inspiring appearance, the Pyramids of Giza served a profound spiritual purpose rooted deeply in ancient Egyptian religious beliefs. While fundamentally monumental tombs for pharaohs like Khufu, Khafre, and Menkaure, they were more than mere burial sites; they were designed to endure an eternity, safeguarding the pharaoh's journey into the afterlife and ensuring their eternal existence.

The distinct pointed shape of the pyramid itself held significant religious symbolism. Historians suggest it was crafted to resemble a "benben," an ancient Egyptian term for a primordial mound of earth from which, in their cosmology, new life was believed to sprout forth. This shape thus connected the pharaoh directly to creation and renewal, acting as a celestial stairway to the heavens. These structures were central to a complex belief system that dictated nearly every aspect of their theocratic society, serving as a physical manifestation of their profound faith in deities and the intricate cosmic order.

### Unveiling Hidden Secrets: Modern Discoveries

Even after millennia, the Giza complex continues to yield startling discoveries, proving that its mysteries are far from exhausted. Recent advancements in scanning technology have allowed researchers to peer into the pyramids' interiors without causing damage, leading to significant revelations.

The ongoing **ScanPyramids project**, a collaboration between researchers from Cairo University and TUM, has been at the forefront of these discoveries. In the Menkaure Pyramid, the third-largest at Giza, the team successfully uncovered two hidden air-filled spaces or void spaces using radar, ultrasound, and electrical resistivity tomography (ERT). This finding provides the first scientific support for a long-held suspicion among archaeologists that an additional entrance might exist on the eastern side of the pyramid, bringing researchers a "big step closer to confirming it."

Furthermore, in 2023, the ScanPyramids project also validated the presence of a "hidden corridor" within the Great Pyramid (Cheops) itself. These findings underscore the intricate and potentially multi-layered internal structures of these ancient edifices. Beyond the known chambers, there have also been sensational claims of "massive underground structures" or even a "mystery new city" beneath Giza, particularly beneath the Pyramid of Khafre. While such headlines can sound like clickbait, they stem from real research exploring subterranean features. The veracity of these claims is still being assessed, but if even part of what has been suggested proves true—such as ancient geological features deliberately built over by the Egyptians—it could fundamentally rewrite history, revealing that the builders chose Giza not just for its surface location, but for what lay hidden beneath.

The Pyramids of Giza remain an unparalleled source of human fascination. Despite centuries of study and groundbreaking modern discoveries, they continue to guard many of their secrets. Each new scan, each unearthed artifact, only deepens our appreciation for the ingenuity of their builders and reinforces the enduring allure of these ancient wonders, forever inviting us to ponder the depths of human history and the mysteries that still lie beneath the sands of time.

**Conclusion and Future RoadMap**

Congratulations!!  You have accomplished a fantastic job of creating a live example of a Human in loop research agent utilizing Gemini 2.5 Flash and LangGraph!

**What Was Learned?**

- **Structured Planning** – Gemini's JSON mode for defining the logic for a dependable acting plan.
- **Cyclical Logic** – Moving from a linear approach to workflows to an adaptive set of workflows in cycles.
- **Human In The Loop** – Included the 'interrupt_before' command and added memory saving functionality to implement a “manager” step for approval cycles.

**Roadmap to Production**

The example presented is a prospective human-in-the-loop research agent (HILRA) application with level 1 functionalities. However, the following will significantly enhance the architectural model for HILRA moving from prototype to a production-ready human-in-the-loop research agent:

1.  **Persistence (Data Storage):**
    - Current : The current version of the Memory Saver expands to use RAM, and as such once the notebook has been shut down/started, the work done will be lost.
    - Future : Implementing a permanent solution to store persistent data and maintain the ability to restart where we left off will be achieved by using a permanent storage solution such as Postgres, Redis, or CheckPoint.
  
2.  **Agentic RAG (Retrieval Augmented Generation):**
    - Current : Currently, the HILRA accesses the internet to search for content to assist the research.
    - Future : By integrating a Vector database with a Vector engine, HILRAs will be able to retrieve all information stored on the private PDFs and produce a response from that information.

3.  **Observability (Using LangSmith):**
    - Current : Currently, we are only able to debug using 'print()' statements. As such, debugging across several papers can be challenging.
    - Future: The integration of LangSmith will allow for visualisation of the full execution paths, token usage on the execution graph, and to verify at what steps the loops fail effectively.

4.  **Collaborative Agents:**
    - Future : The intention is to ultimately have the capability to utilise many specialised HILRAs instead of one (Generalisation) HILRA to enable much broader and deeper research opportunities.